In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.cassandra._
val sparkSession = SparkSession.builder().config(sc.getConf).getOrCreate()
import sparkSession.implicits._

In [3]:
sparkSession.read.textFile("file:///usr/lib/yelp_data/stopwords.txt").createTempView("stop_words")
sparkSession.read.cassandraFormat(keyspace = "yelp", table = "review").load().createTempView("review")
sparkSession.read.cassandraFormat(keyspace = "yelp", table = "business").load().createTempView("business")

In [5]:
sparkSession.sql("""
select /*+ MAPJOIN(stop_words) */
    business_name,
    word,
    avg,
    total
from(
    select 
        distinct 
        b.name business_name,
        trim(regexp_replace(lw.word,"[^a-z]","")) word,
        avg(r.stars) avg,
        sum(r.stars) total
    from review r
         inner join business b on b.business_id = r.business_id
     lateral view explode(split(lower(r.text)," ")) lw as word
    group by r.business_id,b.name,word
) T
left join stop_words sw on sw.value=T.word
where sw.value is null and T.word <> ''
order by T.avg desc,T.total desc
"""
).show(50,false)

+-------------------+----------+---+-----+
|business_name      |word      |avg|total|
+-------------------+----------+---+-----+
|Cut and Taste      |food      |5.0|70   |
|Cut and Taste      |cut       |5.0|60   |
|Cut and Taste      |taste     |5.0|55   |
|Cut and Taste      |jeremy    |5.0|30   |
|Cut and Taste      |menu      |5.0|30   |
|Cut and Taste      |guests    |5.0|25   |
|Innovative Vapors  |best      |5.0|25   |
|Cut and Taste      |wedding   |5.0|25   |
|Innovative Vapors  |know      |5.0|25   |
|Innovative Vapors  |helped    |5.0|25   |
|Innovative Vapors  |vaping    |5.0|20   |
|Innovative Vapors  |lounge    |5.0|20   |
|Cut and Taste      |impressive|5.0|20   |
|Cut and Taste      |received  |5.0|20   |
|Cut and Taste      |catered   |5.0|20   |
|Innovative Vapors  |told      |5.0|20   |
|Innovative Vapors  |excellent |5.0|20   |
|Innovative Vapors  |staff     |5.0|20   |
|Cut and Taste      |best      |5.0|20   |
|Cut and Taste      |party     |5.0|20   |
|Cut and Ta